In [37]:
import json
import numpy as np
from pprint import pprint
fileName = '/Users/erichsu/LING406/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'

data = []
count = 0
with open(fileName) as f:
    for line in f:
        if count<1000000:
            count += 1
            data.append(json.loads(line))

In [3]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
import nltk
from nltk.probability import *

import nltk.data
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
sentences = review_to_sentences(data[1]['text'], tokenizer, True)
fd1 = nltk.FreqDist(sentences[2])

In [6]:
fd1

FreqDist({u'takes': 1, u'get': 1, u'calling': 1, u'hours': 1, u'answer': 1, u'repeated': 1, u'usually': 1})

In [7]:
import pandas as pd
reviews = pd.DataFrame(data)

In [8]:
goodReviews = reviews[reviews['stars']>3].reset_index()
badReviews = reviews[reviews['stars']<=3].reset_index()

In [9]:
def frequencyList(corpus):
    freq = {}
    for i in range(len(corpus)):
        sentences = review_to_sentences(data[i]['text'], tokenizer, True)
        for s in sentences:
            for w in s:
                if freq.has_key(w):
                    freq[w] = freq[w] + 1
                else:
                    freq[w] = 1
    return freq

In [10]:
GoodFreq = frequencyList(goodReviews)

/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://eatinginpittsburgh.com/2011/02/11/coffee-pittsburgh-part-2/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/erichsu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://eatinginpittsburgh.com/2010/07/07/ive-got-an-it

In [11]:
BadFreq = frequencyList(badReviews)

In [12]:
import operator
sorted_g = sorted(GoodFreq.items(), key=operator.itemgetter(1))
sorted_b = sorted(BadFreq.items(), key=operator.itemgetter(1))

In [13]:
sorted_g.reverse()
sorted_b.reverse()

In [14]:
def word_feats(words):
    return dict([(word, True) for word in words])

In [16]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

def word_feats(words):
    return dict([(word, True) for word in words])

In [17]:
goodReview = []
badReview = []
for i in range(len(data)):
    if data[i]['stars']>3:
        goodReview.append(data[i]['text'])
    else:
        badReview.append(data[i]['text'])

In [18]:
posfeats = [(word_feats(review_to_wordlist(f)), 'pos') for f in goodReview]
negfeats = [(word_feats(review_to_wordlist(f)), 'neg') for f in badReview]

In [19]:
negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))

train on 74999 instances, test on 25001 instances


In [34]:
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features(100)

accuracy: 0.508659653614
Most Informative Features
          unprofessional = True              neg : pos    =     57.5 : 1.0
                  argued = True              neg : pos    =     39.7 : 1.0
             inexcusable = True              neg : pos    =     37.7 : 1.0
           disrespectful = True              neg : pos    =     33.7 : 1.0
                 mumbled = True              neg : pos    =     31.5 : 1.0
            unappetizing = True              neg : pos    =     26.8 : 1.0
                   livid = True              neg : pos    =     26.6 : 1.0
              flavorless = True              neg : pos    =     26.1 : 1.0
                    scam = True              neg : pos    =     24.6 : 1.0
                 visibly = True              neg : pos    =     24.1 : 1.0
                  rudest = True              neg : pos    =     24.1 : 1.0
                appalled = True              neg : pos    =     24.1 : 1.0
                  mildew = True              neg 

In [21]:
from nltk.classify.scikitlearn import SklearnClassifier

In [22]:
import numpy as np
from nltk.probability import FreqDist
from nltk.classify import SklearnClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfTransformer()),
                     ('chi2', SelectKBest(chi2, k=1000)),
                     ('nb', MultinomialNB())])
classif = SklearnClassifier(MultinomialNB())

In [23]:
classif.train(trainfeats)

<SklearnClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))>

In [24]:
classifier = classif.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)

accuracy: 0.838446462142


In [36]:
len(GoodFreq)

60961